# Pitch Extraction Simulation notebook 

This notebook is for testing of simulation blocks

In [304]:
import pyaudio
import struct
import matplotlib.pyplot as plt
import matplotlib.animation as animation
# from matplotlib import style
import numpy as np
import sys, select, os
import time
import csv
%matplotlib qt

FORMAT = pyaudio.paInt16
CHANNELS = 1 # Mono audio
SAMPLE_RATE = 44100 # Sample rate
WINDOW_SAMPLES = 2048 # Each window will be 2048 samples long at 22.68us per sample = 0.046s per window
WINDOWS_PER_BUFFER = 20
FRAMES_PER_BUFFER = WINDOW_SAMPLES*WINDOWS_PER_BUFFER

In [305]:
p = pyaudio.PyAudio()
streamIn = p.open(
        format = FORMAT,
        channels = CHANNELS,
        rate = SAMPLE_RATE,
        input = True,
        output = False,
        frames_per_buffer = FRAMES_PER_BUFFER
        )

In [113]:
BB_FORMAT = pyaudio.paInt16
BB_CHANNELS = 1 
BB_SAMPLE_RATE = 44100
BB_WINDOW_SAMPLES = 4096 
BB_WINDOWS_PER_BUFFER = 5
BB_FRAMES_PER_BUFFER = BB_WINDOW_SAMPLES*BB_WINDOWS_PER_BUFFER

BB_p = pyaudio.PyAudio()
BB_streamIn = BB_p.open(format = BB_FORMAT,
                       channels = BB_CHANNELS,
                       rate = BB_SAMPLE_RATE,
                       input = True,
                       output = False,
                       frames_per_buffer = BB_FRAMES_PER_BUFFER)

### AMDF Pitch extractor function

In [125]:
# Function is defined as Dn = 1/L * sum^{L}_{k=0} |S_{j} - S{j-n}| with n = 0,1,2,3... 
#                                                              and Sj are the samples
def amdf_PE(inputWindow):
    D_tau = np.zeros((8,256))
    minIndices = np.zeros(8)
    freq = np.zeros(8)
    vol = np.zeros(8)

    for c in range(8):
        inputWindow_block = inputWindow[c*256:(c+1)*256]
        tau = np.arange(0,(len(inputWindow_block)-1))
        for i,t in enumerate(tau):
            temp = 0
            shifted = np.zeros_like(inputWindow_block)
            if t == 0:
                shifted = inputWindow_block
            else:
                shifted[:t] = 0
                shifted[t:] = inputWindow_block[:-t]

            diffArr = np.abs(np.subtract(inputWindow_block,shifted))
            temp = (np.sum(diffArr)/len(inputWindow_block))
            D_tau[c,i] = temp
            
        offset = np.argmax(D_tau[c,:])
        minIndices[c] = (c*256+offset)+np.argmin(D_tau[c,offset:-1])
        freq[c] = SAMPLE_RATE/(minIndices[c]-(c*256))
        vol[c] = 20*np.log10(np.mean(np.abs(inputWindow))) - 96.3
    
    return D_tau.flatten(),minIndices, freq, vol

In [306]:
def amdf_PE_2(inputWindow):
    D_tau = np.zeros((8,256))
    minIndices = np.empty(8)
    freq = np.empty(8)
    vol = np.empty(8)
    tau = np.arange(1,255)
    
    for c in range(8):       
        inputWindow_block = inputWindow[c*5120:(c*5120)+256]    
        for i,t in enumerate(tau, start = 1):
            shifted = np.zeros_like(inputWindow_block)
            shifted[t:] = inputWindow_block[:-t]
            D_tau[c,i] = np.sum(np.abs(inputWindow_block-shifted))/256

        offset = np.argmax(D_tau[c,:])
        minIndices[c] = (c*256+offset)+np.argmin(D_tau[c,offset:-1])
        freq[c] = (44100/(minIndices[c]-(c*256)))
        vol[c] = 20*np.log10(np.mean(np.abs(inputWindow))) - 96.3
    
    return D_tau, minIndices, freq, vol

In [38]:
from sklearn.preprocessing import normalize
x = np.random.rand(100)

norm2 = normalize(x[:,np.newaxis], axis = 0).ravel()
plt.plot(x)
plt.plot(norm1)
plt.plot(norm2)

In [86]:
a = np.array([19,2,13,45,5,6,7,28,9,5,5,5,31,11,1,52,5,5])
test = np.mean(a)
test

14.11111111111111

### Stretch function
Trying to determine an on-the-fly function generator that will compensate for the decay of the AMDF. The linear function does not work as the decay is not linear and approximation with a linear function does not yield good results yet

Possible fixes:
    - scaling factor in the denominator
    - non-linear function (will require some form of interpolation)

In [6]:
## Stretch function (not currently working)
D_max = np.max(D_tau)
stretch = np.zeros(FRAMES_PER_BUFFER)
stretch[0] = 1
for n in range(1,FRAMES_PER_BUFFER):
    stretch[n] = D_max*((-1/(10*2*FRAMES_PER_BUFFER))*n)
D_tau = (D_tau*stretch-(D_max/2))

NameError: name 'D_tau' is not defined

In [43]:
plt.plot(stretch)

In [7]:
data = streamIn.read(FRAMES_PER_BUFFER)
data_int = np.frombuffer(data,dtype='<i2')
# print(data_int)
a = time.time()
AMDF_data = amdf_PE(data_int)
b = time.time()
print("Time taken: {}".format(b-a))
# plt.plot(AMDF_data)

Time taken: 0.026920795440673828


# Plotting function

### PC simulation:
Initial PC simulation for PoC and AMDF design

In [307]:
fig, ax = plt.subplots()
ax.set_title("AMDF")
ax.set_ylim([-(2**8),(2**14)])
vline0 = ax.axvline(x =   0.,color = 'red')
vline01 = ax.axvline(x =   0.,color = 'red')
vline1 = ax.axvline(x = 256,color = 'k')
vline01 = ax.axvline(x = 256,color = 'k')
vline2 = ax.axvline(x = 512,color = 'green')
vline02 = ax.axvline(x = 512,color = 'green')
vline3 = ax.axvline(x = 768,color = 'orange')
vline03 = ax.axvline(x = 768,color = 'orange')
vline4 = ax.axvline(x = 1024,color = 'pink')
vline04 = ax.axvline(x = 1024,color = 'pink')
vline5 = ax.axvline(x = 1280,color = 'purple')
vline05 = ax.axvline(x = 1280,color = 'purple')
vline6 = ax.axvline(x = 1536,color = 'yellow')
vline06 = ax.axvline(x = 1536,color = 'yellow')
vline7 = ax.axvline(x = 1792,color = 'cyan')
vline07 = ax.axvline(x = 1792,color = 'cyan')
line, = ax.plot(np.zeros(2048))

# line1, = ax.plot(np.zeros(FRAMES_PER_BUFFER))
# line2, = ax.plot(np.zeros(FRAMES_PER_BUFFER))
plt.show()
signal = []
amdf_val = []
stretch = []
freqs = []
vols = []
func_time = []
timer = 0
num_frames = 0
timerStart = time.time()
while num_frames<50:
    data = streamIn.read(FRAMES_PER_BUFFER)
    data_int2 = np.frombuffer(data,dtype='<i2')
#     print(len(data_int2))
    timer = time.time()-timerStart
    func_time_start = time.time()
    amdf_data, vline, freqArr, vol = amdf_PE_2(data_int2)
    func_time.append(time.time()-func_time_start)
    line.set_ydata(amdf_data.flatten())
    freqs.append(freqArr)
    vols.append(vol)
#     if timer>= 3.0:
#         finalTimer = time.time()-timerStart
#         break
    vline0.set_data(vline[0] , [0,1])
    vline1.set_data(vline[1] , [0,1])
    vline2.set_data(vline[2] , [0,1])
    vline3.set_data(vline[3] , [0,1])
    vline4.set_data(vline[4] , [0,1])
    vline5.set_data(vline[5] , [0,1])
    vline6.set_data(vline[6] , [0,1])
    vline7.set_data(vline[7] , [0,1])
    fig.canvas.draw()
    fig.canvas.update()
    fig.canvas.flush_events()
    num_frames += 1
# plt.close(fig)
freqs = np.array(freqs)
freqs = freqs.flatten()
vols = np.array(vols)
vols = vols.flatten()
# vols /= np.max(vols) 
print(timer)
xAxis = np.round(np.arange(0,timer,timer/8),2)
xticker = np.arange(len(freqs))
fig, ax1 = plt.subplots()
fig.suptitle("Frequency and volume progression of recorded audio")
ax1.set_ylabel("Frequency (Hz)", color = "green")
ax1.set_xlabel("Time (sampled at {} Hz)".format(SAMPLE_RATE))
ax1.plot(freqs, color = 'green')

ax2 = ax1.twinx()
ax2.set_ylabel("Volume (dB)", color = "red")
ax2.plot(vols, color = 'red')
plt.xticks(xticker[::66], xAxis)

45.276118993759155


([<matplotlib.axis.XTick at 0x1a8c0f504a8>,
 [Text(0, 0, '0.0'),
  Text(0, 0, '5.66'),
  Text(0, 0, '11.32'),
  Text(0, 0, '16.98'),
  Text(0, 0, '22.64'),
  Text(0, 0, '28.3'),
  Text(0, 0, '33.96')])

### BBB Simulation:
Simulating the AMDF for the BeagleBone Black recording speed limitations

In [120]:
fig, ax = plt.subplots()
ax.set_title("AMDF")
ax.set_ylim([-(2**4),(2**12)])
vline0 = ax.axvline(x =   0.,color = 'red')
vline01 = ax.axvline(x =   0.,color = 'red')
vline1 = ax.axvline(x = 256,color = 'k')
vline01 = ax.axvline(x = 256,color = 'k')
vline2 = ax.axvline(x = 512,color = 'green')
vline02 = ax.axvline(x = 512,color = 'green')
vline3 = ax.axvline(x = 768,color = 'orange')
vline03 = ax.axvline(x = 768,color = 'orange')
vline4 = ax.axvline(x = 1024,color = 'pink')
vline04 = ax.axvline(x = 1024,color = 'pink')
vline5 = ax.axvline(x = 1280,color = 'purple')
vline05 = ax.axvline(x = 1280,color = 'purple')
vline6 = ax.axvline(x = 1536,color = 'yellow')
vline06 = ax.axvline(x = 1536,color = 'yellow')
vline7 = ax.axvline(x = 1792,color = 'cyan')
vline07 = ax.axvline(x = 1792,color = 'cyan')
line, = ax.plot(np.zeros(FRAMES_PER_BUFFER))

# line1, = ax.plot(np.zeros(FRAMES_PER_BUFFER))
# line2, = ax.plot(np.zeros(FRAMES_PER_BUFFER))
plt.show()
BB_signal = []
BB_amdf_val = []
BB_stretch = []
BB_freqs = []
BB_vols = []
BB_func_time = []
timer = 0
num_frames = 0
timerStart = time.time()
while num_frames<56:
    BB_data = BB_streamIn.read(BB_FRAMES_PER_BUFFER)
    BB_data_int2 = np.frombuffer(BB_data,dtype='<i2')
#     print(len(data_int2))
    timer = time.time()-timerStart
    bb_func_timer_start = time.time()
    amdf_data, vline, freqArr, vol = amdf_PE_2(BB_data_int2)
    BB_func_time.append(time.time() - bb_func_timer_start)
    line.set_ydata(amdf_data.flatten())
    BB_freqs.append(freqArr)
    BB_vols.append(vol)
#     if timer>= 3.0:
#         finalTimer = time.time()-timerStart
#         break
    vline0.set_data(vline[0] , [0,1])
    vline1.set_data(vline[1] , [0,1])
    vline2.set_data(vline[2] , [0,1])
    vline3.set_data(vline[3] , [0,1])
    vline4.set_data(vline[4] , [0,1])
    vline5.set_data(vline[5] , [0,1])
    vline6.set_data(vline[6] , [0,1])
    vline7.set_data(vline[7] , [0,1])
    fig.canvas.draw()
    fig.canvas.update()
    fig.canvas.flush_events()
    num_frames += 1
# plt.close(fig)
BB_freqs = np.array(BB_freqs)
BB_freqs = BB_freqs.flatten()
BB_vols = np.array(BB_vols)
BB_vols = BB_vols.flatten()
# vols /= np.max(vols) 
print(timer)
xAxis = np.round(np.arange(0,timer,timer/8),2)
xticker = np.arange(len(BB_freqs))
fig, ax1 = plt.subplots()
fig.suptitle("Frequency and volume progression of recorded audio")
ax1.set_ylabel("Frequency (Hz)", color = "green")
ax1.set_xlabel("Time (sampled at {} Hz)".format(BB_SAMPLE_RATE))
ax1.plot(BB_freqs, color = 'green')

ax2 = ax1.twinx()
ax2.set_ylabel("Volume (dB)", color = "red")
ax2.plot(BB_vols, color = 'red')
plt.xticks(xticker[::66], xAxis)

25.30523133277893


([<matplotlib.axis.XTick at 0x1a8b06e1128>,
 [Text(0, 0, '0.0'),
  Text(0, 0, '3.16'),
  Text(0, 0, '6.33'),
  Text(0, 0, '9.49'),
  Text(0, 0, '12.65'),
  Text(0, 0, '15.82'),
  Text(0, 0, '18.98')])

In [308]:
plt.plot(func_time)
print(np.mean(func_time))

0.022172150611877443


In [121]:
plt.plot(BB_func_time)

### Writing results to file

In [29]:
with open('frequency_list.csv','w') as file1:
    np.savetxt(file1, freqs, fmt = '%10.3f', delimiter = ',')

with open('volume_list.csv','w') as file2:
    np.savetxt(file2, vols, fmt= '%10.3f', delimiter = ',')

In [82]:
xAxis

array([0.  , 0.38, 0.75, 1.13, 1.5 , 1.88, 2.26, 2.63])

## Plotting BBBW output for accuracy and speed testing

In [12]:
print(vol)

-41.88787574978075
